In [1]:
# ugly fix for the shadow of the map widgets
from IPython.display import display, HTML

display(HTML("<style>.leaflet-widgetcontrol {box-shadow: none}</style>"))

In [2]:
from sepal_ui import sepalwidgets as sw
from component.message import cm

Styles()

ResizeTrigger()

In [3]:
# create an appBar
rp_appBar = sw.AppBar(cm.app.title, translator=cm)

In [ ]:
# add tiles
%run 'aoi_ui.ipynb'
%run 'questionnaire_ui.ipynb'
%run 'compute_ui.ipynb'
%run 'about_ui.ipynb'

rp_content = [
    rp_aoi,
    rp_questionnaire,
    rp_validation,
    rp_map_tile,
    rp_regions,
    rp_theme,
    rp_carbon,
    rp_about,
    rp_disclaimer,
]

In [ ]:
# create a drawer
items = [
    sw.DrawerItem(cm.app.drawer.about, "mdi-help-circle", card="about_tile"),
    sw.DrawerItem(cm.app.drawer.aoi, "mdi-map-marker-check", card="aoi_tile"),
    sw.DrawerItem(
        cm.app.drawer.question, "mdi-file-question", card="questionnaire_widget"
    ),
    sw.DrawerItem(cm.app.drawer.recipe, "mdi-cogs", card="compute_widget"),
    sw.DrawerItem(cm.app.drawer.map, "mdi-map", card="map_widget"),
    sw.DrawerItem(
        cm.app.drawer.dashboard, "mdi-view-dashboard", card="dashboard_widget"
    ),
]

code_link = "https://github.com/sepal-contrib/se.plan"
wiki_link = "https://docs.sepal.io/en/latest/modules/dwn/seplan.html"
issue_link = "https://github.com/sepal-contrib/se.plan/issues/new"

rp_drawer = sw.NavDrawer(items, code=code_link, wiki=wiki_link, issue=issue_link)

In [ ]:
# build the app
rp_app = sw.App(tiles=rp_content, appBar=rp_appBar, navDrawer=rp_drawer, translator=cm)

In [ ]:
# display the app
rp_app.show_tile("about_tile").add_banner(cm.app.banner)